1. Inicialización e importaciones

In [6]:
import sys
import os

# FIX para el ModuleNotFoundError
if 'notebooks' in os.getcwd():
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
else:
    sys.path.append(os.path.abspath(os.getcwd()))

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

# Importar funciones
from src.data_loader import load_fakenewsnet_data_from_csv, extract_text_from_url
from src.processing import clean_text

# Configurar pandas
pd.set_option('display.max_colwidth', 200)
print("Inicialización completa.")

Inicialización completa.


2. Cargar y preprocesamiento

In [8]:
# FIX CLAVE: Asegura que el directorio de salida exista 
processed_dir = 'data/processed'
# También revisamos y creamos el directorio para las gráficas de resultados
results_dir = 'results/figures' 

# Crea la carpeta data/processed
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)
    print(f"Directorio creado: {processed_dir}")

# Crea la carpeta results/figures 
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    print(f"Directorio creado: {results_dir}")


# 1. Cargar los datos
df = load_fakenewsnet_data_from_csv()

# Solo mantener los artículos que tienen texto y URL válidos 
df = df[df['text'] != ''].reset_index(drop=True)
print(f"Total de artículos listos para procesamiento: {len(df)}") 

# 2. Limpiar el texto del artículo
print("\nLimpiando texto de artículos")
df['cleaned_text'] = df['text'].apply(clean_text)

# 3. Extraer y limpiar el texto de las imágenes (OCR)
print("Ejecutando OCR en imágenes")
df['ocr_text'] = df['image_url'].apply(extract_text_from_url) 
df['cleaned_ocr_text'] = df['ocr_text'].apply(clean_text)

# 4. Crear nuestras dos entradas de texto
df['X_baseline'] = df['cleaned_text']
df['X_ocr_model'] = df['cleaned_text'] + " " + df['cleaned_ocr_text']

# 5. Definir nuestra variable objetivo (y)
y = df['label']

print("\nMuestra de datos procesados")
print(df[['X_baseline', 'X_ocr_model', 'label']].head(2))

# 6. Guardar los datos procesados 
df.to_csv('data/processed/fakenewsnet_processed.csv', index=False)
print(f"\nDatos procesados guardados: {len(df)} artículos.")

Total de artículos listos para procesamiento: 0

Limpiando texto de artículos
Ejecutando OCR en imágenes

Muestra de datos procesados
Empty DataFrame
Columns: [X_baseline, X_ocr_model, label]
Index: []

Datos procesados guardados: 0 artículos.
